In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [53]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-11-11/"
files_start = 2336
files_end = 2337


In [54]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [55]:
filenames = os.listdir(path_all)

In [56]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [57]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2015-11-12 18:28:00,1,blank sol,2.902020e-05,0.006918,0.012085,0.017667,0.000016,0.000857,0.004367,0.000654,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,2,blank sol,2.819000e-05,0.006907,0.012048,0.017567,-0.000027,0.000288,0.003157,0.000204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,3,blank sol,2.019260e-05,0.006875,0.012049,0.017517,0.000016,0.000280,0.002931,0.000196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,4,blank sol,3.166470e-05,0.006920,0.012080,0.017597,-0.000013,0.000345,0.003030,0.000243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,5,blank sol,2.964180e-05,0.006868,0.012020,0.017551,0.000048,0.000328,0.003046,0.000214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,6,blank sol,1.206050e-05,0.006878,0.011978,0.017495,-0.000001,0.000237,0.003012,0.000222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,7,blank sol,-3.012400e-06,0.006893,0.012044,0.017545,0.000030,0.000261,0.002984,0.000174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,8,blank sol,3.755320e-05,0.006901,0.012074,0.017560,0.000051,0.000298,0.003018,0.000226,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-11-12 18:28:00,9,blank sol,8.481900e-06,0.006910,0.012065,0.017573,0.000017,0.000293,0.002971,0.000217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2015-11-11 14:17:00,0.000021,0.006030,0.011371,0.037035,3.431785e-06,0.000287,0.002224,0.000193,0.002912,0.001843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 15:07:00,0.000015,0.006100,0.011456,0.035200,8.553200e-06,0.000535,0.002961,0.000398,0.003877,0.003234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 17:26:00,0.000012,0.006018,0.011303,0.032058,9.235880e-06,0.000293,0.002250,0.000178,0.002934,0.001801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 18:16:00,0.000018,0.006049,0.011341,0.030306,3.673875e-06,0.000499,0.003090,0.000363,0.004030,0.002999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 19:08:00,0.000005,0.006200,0.011651,0.029370,9.965350e-06,0.000358,0.002768,0.000252,0.003630,0.002250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 19:59:00,0.000023,0.006396,0.011940,0.028377,3.023505e-06,0.000396,0.002835,0.000268,0.003717,0.002449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 20:50:00,0.000010,0.006491,0.012045,0.026963,8.890815e-06,0.000302,0.002642,0.000213,0.003423,0.002039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 21:39:00,0.000004,0.006446,0.011881,0.025896,-1.987620e-06,0.000555,0.003173,0.000404,0.004144,0.003303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol
2015-11-11 22:29:00,0.000010,0.006499,0.011957,0.025267,1.317012e-05,0.000419,0.002953,0.000298,0.003856,0.002650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blank sol


In [11]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [59]:
df_all_mean.to_csv(path_sav + "blanks_mean.csv", mode='a', header=False)

In [215]:
df_internal_sub.to_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm.csv", mode='a', header=False)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")